In [1]:
import cv2
import numpy as np

def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size): # 가우시안 필터 ==> 이미지에서 노이즈 제거
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold): # Canny 알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def ROI(img,vertices,ignore_color): #관심영역 설정
    mask = np.zeros_like(img) #같은 크기의 검은색으로 채운 이미지
    cv2.fillPoly(mask, vertices, ignore_color)#다각형 그리기(vertices = 다각형 좌표)
    roi_image = cv2.bitwise_and(img,mask)
    return roi_image

def draw_lines(img, lines): #선 그리기
    color = [0,255,0]
    thickness = 2
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1,y1), (x2,y2), color, thickness)

#허프라인 그리기
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength = min_line_len,
                            maxLineGap = max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3) , dtype = np.uint8)
    #draw_lines(line_img, lines)
    return lines #라인만 반환



def masking(img, initial_img): #실제 영상에 선 입히기
    a = 2
    b = 1.
    c = 0.
    return cv2.addWeighted(initial_img, a, img, b, c)

def get_fitline(img, f_lines): # 대표선 구하기   
    lines = np.squeeze(f_lines)
    if(len(lines.shape) == 1): #선이 하나밖에 없는 경우 배열의 모양 따로 조정
        lines = lines.reshape(2,2)
    else:
        lines = lines.reshape(lines.shape[0]*2,2)
    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2-200)-y)/vy*vx + x) , int(img.shape[0]/2-200)
    
    result = [x1,y1,x2,y2]
    return result

def draw_fit_line(img, lines, color=[0, 0, 255], thickness=10): # 대표선 그리기
        cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness)
        
def expression(x1,y1,x2,y2,x3,y3,x4,y4):
    m_a = (y2 - y1) / (x2 -x1) #legt line 기울기
    m_b = (y4 - y3) / (x4 - x3) #right line 기울기
    n_a = -(m_a * x1 ) + y1
    n_b = -(m_b * x3 ) + y3
    x = (n_b - n_a) / (m_a - m_b) 
    y = m_a * ((n_b - n_a) / (m_a - m_b)) + n_a 
    return x,y




#동영상 파일 받아오기
video_name = '테스트영상3.mp4'
capture = cv2.VideoCapture(video_name)
if capture.isOpened():
    print('width: {}, height: {}'.format(capture.get(3), capture.get(4)))

#허프라인 검출 설정
rho = 1 #원점에서 직선까지의 거리를 얼마씩 증가시키면서 조사할것인지
theta = np.pi/180
threshold = 30 #직선이라고 판단하기 위한 최소 교차점(hough space에서)
min_line_len = 10 #하나의 선이라 판단하는 최소 길이
max_line_gap = 20 #선위의 점들 사이의 최대 길이(이거 넘어가면 다른 선이라 판단)



#영상 재생 및 허프라인 변환
while(capture.isOpened()):
    ret, frame = capture.read() #열기
    height, width = frame.shape[:2] #프레임의 높이 너비
    m_width = (int)(width/2)
    #vertices (관심영역 설정), 채우는 색
    vertices = np.array([[(350,height-150),
                          (600,360),
                          (800,360),
                          (width-100,height-150)]], dtype=np.int32)
    fill_color = 255
    
    height, width = frame.shape[:2] # 이미지 높이, 너비
    gray_img = grayscale(frame)     # 흑백이미지로 변환
    blur_img = gaussian_blur(gray_img, 5) # Blur 효과
    canny_img = canny(blur_img,70,200) #선 따기
    roi_img = ROI(canny_img,vertices,fill_color) #관심영역 설정
    hough_img = hough_lines(roi_img, rho, theta, threshold, min_line_len, max_line_gap) #검은 배경에 라인 따기

    hough_img = np.squeeze(hough_img)#squeeze ==> 차원 줄이기(axis 값 안주면 1인값 제거) [8,1,4]

    slope_degree = (np.arctan2(hough_img[:,1] - hough_img[:,3], hough_img[:,0] - hough_img[:,2]) * 180) / np.pi
    # 수평 기울기 제한
    hough_img = hough_img[np.abs(slope_degree)<165]
    slope_degree = slope_degree[np.abs(slope_degree)<165]
    # 수직 기울기 제한
    hough_img = hough_img[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
    # 필터링된 직선 버리기
    L_lines, R_lines = hough_img[(slope_degree>0),:], hough_img[(slope_degree<0),:]
    temp = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]
    
    if(len(L_lines) == 0 and len(R_lines) == 0): #L_lines, R_lines 모두 없는 경우
        L_lines = pre_left_line
        R_lines = pre_right_line
    elif(len(L_lines) == 0):#L_lines만 없는 경우
        L_lines = pre_left_line
        pre_right_line = R_lines
    elif(len(R_lines) == 0):#R_lines만 없는 경우
        R_lines = pre_right_line
        pre_left_line = L_lines
    else:#라인 모두 검출한 경우
        pre_right_line = R_lines
        pre_left_line = L_lines
    
    # 왼쪽, 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(frame,L_lines)
    right_fit_line = get_fitline(frame,R_lines)
    # 대표선 그리기
    draw_fit_line(temp, left_fit_line)
    draw_fit_line(temp, right_fit_line)
    #소실점 구하기
    vanishing_point = expression(left_fit_line[0],left_fit_line[1],left_fit_line[2],left_fit_line[3],right_fit_line[0],right_fit_line[1],right_fit_line[2],right_fit_line[3])
    v_x = int(vanishing_point[0])
    v_y = int(vanishing_point[1])
    
    result_img = masking(temp,frame) #실제 영상에 대표선 넣기
    
    cv2.circle(result_img, (v_x,v_y) , 6,(0,0,255),-1) #소실점
    cv2.line(result_img,(m_width,0),(m_width,300),(255,255,0),5) #프레임 중앙 라인

    if(v_x > m_width): #중앙보다 오른쪽이면 right
        print("Right!!!")
        cv2.circle(result_img,(1000,50), 6,(0,0,255),-1)

    if(v_x < m_width):
        print("Left!!!")
        cv2.circle(result_img,(100,50), 6,(0,0,255),-1)

    
    cv2.imshow("resultFrame",result_img)

    if cv2.waitKey(33) & 0xFF == ord('q'):#q누르면 창 끄기, 1ms마다 프레임 재생
        break

capture.release()
#영상 중지 됨 ==> 메모리 프리
cv2.destroyAllWindows()
#이미지 창 모두 끄기

width: 1280.0, height: 720.0
Right!!!
Right!!!
Right!!!
Left!!!
Right!!!
Right!!!
Right!!!
Left!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Left!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Left!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Right!!!
Left!!!
Left!!!
Right!!!
Right!!!
Left!!!
Left!!!
Right!!!
Left!!!
Left!!!
Left!!!
Left!!!
Right!!!
Right!!!
Right!!!
Left!!!
Right!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Right!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Right!!!
Right!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!
Left!!!